In [1]:
import wandb
from tqdm import tqdm
import numpy as np

api = wandb.Api()

def load_synthetic_fixed_cost_runs(acq, tfn, dim):
    runs = api.runs(path="ziv-scully-group/PandoraBayesOPT", filters={
        "sweep": "24hhbpyc",
        "config.dim": dim,
        "config.problem": tfn,
        "config.policy": acq})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        summary_metric_keys = ["global optimum value"]
        summary_history = run.scan_history(keys = summary_metric_keys, page_size=1_000_000_000)
        metrics.update({k: [d[k] for d in summary_history] for k in summary_metric_keys})
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [2]:
acquisition_functions = {
    'RandomSearch': 'RandomSearch',
    'ExpectedImprovement':'ExpectedImprovement',
    'ThompsonSampling':'ThompsonSampling', 
    'KnowledgeGradient':'KnowledgeGradient',
    'MultiStepLookaheadEI':'MultiStepLookaheadEI',
    'Gittins_Lambda_01':'Gittins_Lambda01',
    'Gittins_Lambda_001':'Gittins_Lambda001',
    'Gittins_Lambda_0001':'Gittins_Lambda0001',
    'Gittins_Step_Divide2':'Gittins_Step_Divide2',
    'Gittins_Step_Divide5':'Gittins_Step_Divide5',
    'Gittins_Step_Divide10':'Gittins_Step_Divide10',
    'Gittins_Step_EIpu':'Gittins_Step_EIPC',
    }
target_functions_with_optima = {
    'Ackley':0,
    # 'DropWave_2D':-1,
    # 'Shekel5_4D':-10.1532,
    'Rosenbrock':0,
    'Levy':0,
  }
dimensions = [4, 8, 16]

In [3]:
grouped_runs = {(a,t,d): load_synthetic_fixed_cost_runs(a,t,d) for a in acquisition_functions.keys() for t in target_functions_with_optima.keys() for d in (dimensions)}

100%|███████████████████████████████████████████| 16/16 [00:16<00:00,  1.05s/it]


In [12]:
for a in acquisition_functions.keys():
    for (t,opt) in target_functions_with_optima.items():
        for d in dimensions:
            config_and_metrics_per_seed = grouped_runs[a,t,d]
    
            cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0]).T 
            best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed if len(m['best observed'])>0]).T
            
            regret_per_seed = best_observed_per_seed - opt
    
            print(a, t, d, regret_per_seed.shape)
    
            regret_25 = np.quantile(regret_per_seed, 0.25, axis=1)
            regret_50 = np.quantile(regret_per_seed, 0.5, axis=1)
            regret_75 = np.quantile(regret_per_seed, 0.75, axis=1)
    
            output = np.stack((cumulative_cost_per_seed.mean(axis=1), regret_25, regret_50, regret_75),axis=-1)
    
            np.savetxt(f"results/preprint/synthetic/Synthetic_FixedCost_{t}_d{d}_{acquisition_functions[a]}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')

RandomSearch Ackley 4 (41, 16)
RandomSearch Ackley 8 (81, 16)
RandomSearch Ackley 16 (161, 16)
RandomSearch Rosenbrock 4 (41, 16)
RandomSearch Rosenbrock 8 (81, 16)
RandomSearch Rosenbrock 16 (161, 16)
RandomSearch Levy 4 (41, 16)
RandomSearch Levy 8 (81, 16)
RandomSearch Levy 16 (161, 16)
ExpectedImprovement Ackley 4 (41, 16)
ExpectedImprovement Ackley 8 (81, 16)
ExpectedImprovement Ackley 16 (161, 16)
ExpectedImprovement Rosenbrock 4 (41, 16)
ExpectedImprovement Rosenbrock 8 (81, 16)
ExpectedImprovement Rosenbrock 16 (161, 16)
ExpectedImprovement Levy 4 (41, 16)
ExpectedImprovement Levy 8 (81, 16)
ExpectedImprovement Levy 16 (161, 16)
ThompsonSampling Ackley 4 (41, 16)
ThompsonSampling Ackley 8 (81, 16)
ThompsonSampling Ackley 16 (161, 16)
ThompsonSampling Rosenbrock 4 (41, 16)
ThompsonSampling Rosenbrock 8 (81, 16)
ThompsonSampling Rosenbrock 16 (161, 16)
ThompsonSampling Levy 4 (41, 16)
ThompsonSampling Levy 8 (81, 16)
ThompsonSampling Levy 16 (161, 16)
KnowledgeGradient Ackley 4 (